## Dependencies

In [1]:
# load required libraries and utilities.py functions
import os
import pandas as pd
from tifffile import tifffile

from utilities import *

## Directory paths and parameters

In [2]:
# location of images to segment (directory will be searched recursively)
input_path = "~/Research/Images"
match = ".tif"   # search criteria for images (e.g. ".tif")

# output location for segmentation CSVs
output_path = "~/Research/Images"

# nuclear stain position (0 -> first layer, 1 -> second layer, etc.)
nuc_name = "dapi"
nuc_index = 2

# indicate channels (beyond cell nuclei stain) and their channel positions
channels = {"gfp": 1, "rfp": 0}
bit_depth = 16    # change to 8 if your image intensity maximum is 255

## Run segmentation pipeline

In [3]:
# make sure paths are absolute
input_path = abs_path(input_path)
output_path = abs_path(output_path)

# get all images in input directory
image_paths = get_images(input_path, match)

# run the segmentation calculation on each image
for path in image_paths:
    
    # read the image
    image_path = os.path.join(input_path, path)
    image = tifffile.imread(image_path)

    # get the nuclear stain channel and segmen the image based on this
    nuc_channel = image[nuc_index]
    nuc_seg = segmentation(nuc_channel)

    # create extension-less path to image specific file and CSV file path
    out_no_ext = os.path.join(output_path, os.path.splitext(path)[0])
    csv_path = out_no_ext + ".csv"

    # make sure directories exist for file(s)
    dir_path = os.path.split(out_no_ext)[0]
    os.makedirs(dir_path, exist_ok=True)

    # return the detected objects that are identified as cells
    x_pos, y_pos, cells = identify_cells(nuc_seg)

    # get the nuclear intensities and generated image
    nuc_intensities, out_image = overlay_cells(nuc_channel, cells)
    nuc_path = out_no_ext + "-" + nuc_name + ".png"
    # plot(out_image, nuc_path, bit_depth)    # save the image

    # create dictionary for storing cell specific values
    data = {"X": x_pos, "Y": y_pos, nuc_name: nuc_intensities}

    # iterate through the channels, quantifying signal in each
    for sig_name in channels.keys():
        # get expression channel from image
        index = channels[sig_name]
        channel = image[index]

        # calculate intensities
        intensities, out_image = overlay_cells(channel, cells)
        data[sig_name] = intensities
        sig_path = out_no_ext + "-" + sig_name + ".png"
        # plot(out_image, sig_path, bit_depth)    # save the image

    # save cell locations and signal intensities to CSV
    df = pd.DataFrame(data)
    df.to_csv(csv_path, index=False)